# Pre-process DecryptM Dataset

**Publication**: Jana Zecha et al. Decrypting drug actions and protein modifications by dose- and time-resolved proteomics.

In [1]:
import pandas as pd
import os
import toml
import re
import requests
from unipressed import IdMappingClient
import time

## Dose-Dependent Drugs

In [2]:
def search_files(directory):
    """
    Searches for .txt and .toml files in the given directory and its subdirectories.
    Args:
        directory (str): The path to the directory to search in.
    Returns:
        tuple: A tuple containing two lists:
            - txt_files (list): A list of .txt files found.
            - toml_files (list): A list of .toml files found.
    """
    txt_files = []
    toml_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                txt_files.append(file)
            elif file.endswith('.toml'):
                toml_files.append(file)

    return txt_files, toml_files


def process_experiment_column(df):
    """
    Processes the 'Experiment' column in the given DataFrame by splitting it into multiple new columns.
    Args:
        df (pd.DataFrame): The input DataFrame containing an 'Experiment' column to be processed.
    Returns:
        pd.DataFrame: The DataFrame with the 'Experiment' column split into new columns:
            - 'Type'
            - 'Cell_line'
            - 'Drug'
            - 'Time'
            - 'Replicate'
    """
    new_columns = ["Type", "Cell_line", "Drug", "Time", "Replicate"]
    split_columns = df['Experiment'].str.split('_', expand=True)
    num_parts = split_columns.shape[1]
    
    for col in new_columns:
        df[col] = 'NA'
    
    df[new_columns[0]] = split_columns[0]
    df[new_columns[1]] = split_columns[1]
    df[new_columns[2]] = split_columns.iloc[:, 2:num_parts-2].apply(lambda x: '_'.join(x), axis=1)
    df[new_columns[3]] = split_columns[num_parts-2]
    df[new_columns[4]] = split_columns[num_parts-1]
    
    df['Time'] = df['Time'].apply(lambda x: int(x[:-1]) * 60 if x.endswith('h') else int(x[:-3]) if x.endswith('min') else x)  # Convert hours to minutes and make column numeric
    
    df = df.loc[:, ~df.columns.duplicated()]
    df = df[new_columns + [col for col in df.columns if col not in new_columns]]
    
    return df


def update_dataframe_with_toml(df, toml_file, verbose = False):
    """
    Updates the column names of a DataFrame based on information from a TOML file.
    This function reads a TOML file to get dose and channel information, then renames
    the columns of the DataFrame accordingly. The columns to be renamed are expected
    to follow the pattern "TMT Channel Ratio {channel}", and they will be renamed to
    "Dose {dose}" based on the corresponding dose for each channel.
    Args:
        df (pandas.DataFrame): The DataFrame whose columns are to be renamed.
        toml_file (str): The path to the TOML file containing dose and channel information.
    Returns:
        pandas.DataFrame: The DataFrame with updated column names.
    """
    with open(toml_file, 'r') as file:
        toml_data = toml.load(file)
    doses = toml_data['TMT']['doses']
    channels = toml_data['TMT']['channels']
    dose_scale = toml_data['TMT']['dose_scale']
    dose_label = toml_data['TMT']['dose_label']
    
    if dose_scale != '1e-9' and dose_label == 'M':
        if verbose == True:
            print(f"Warning: Dose scale is {dose_scale} and dose label is {dose_label}. Multiplying doses by {float(dose_scale) / 1e-9} to convert to nM.")
        doses = [dose * float(dose_scale) / 1e-9 for dose in doses]
        
    for i, channel in enumerate(channels):
        old_column_name = f"TMT Channel Ratio {channel}"
        if old_column_name in df.columns:
            new_column_name = f"Dose {doses[i]}" if i < len(doses) else f"Dose NA"
            df.rename(columns={old_column_name: new_column_name}, inplace=True)
    return df


def filter_data(df, min_score_cutoff = 60, max_pep_cutoff = 0.05, verbose = False):
    """
    Filters the DataFrame based on the given cutoffs.
    
    Args:
        df (pd.DataFrame): The input DataFrame to be filtered.
        max_score_cutoff (float): The maximum score cutoff for filtering.
        min_pep_cutoff (float): The minimum PEP cutoff for filtering.
    
    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    rows_before = df.shape[0]

    filtered_df = df[
        (df['Max Score'] >= min_score_cutoff) & 
        (df['Min PEP'] <= max_pep_cutoff) & 
        (df['Phospho (STY)'] >= 1) & 
        (df['Phosphoproteome'] == True)
    ]

    rows_after = filtered_df.shape[0]
    if verbose == True:
        print(f"Number of rows before filtering: {rows_before}")
        print(f"Number of rows after filtering: {rows_after}")
    
    return filtered_df


def remove_columns(df):
    """
    Remove specific columns from a DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame from which columns will be removed.
    
    Returns:
        pd.DataFrame: The DataFrame with the specified columns removed.
    """
    columns_to_remove = [
        'Type', 'Replicate', 'N duplicates', 'Sequence', 'Length',
        'Missed cleavages', 'Proteins', 'Leading proteins', 'Protein names',
        'Phospho (STY)', 'All Phospho (STY) Probabilities', 'Max Score', 'Min PEP',
        'Intensity', 'Phosphoproteome', 'Fullproteome', 'Curve signal', 'Log EC50',
        'Curve slope', 'Curve top', 'Curve bottom', 'R2', 'Curve RMSE', 'Log EC50 error',
        'Curve slope error', 'Curve top error', 'Curve bottom error', 'EC50', 'pEC50',
        'Curve effect size', 'Regulation', 'Acetyl (K)', 'All Acetyl (K) Probabilities', 'Acetylproteome'
    ]

    df.drop(columns=columns_to_remove, inplace=True, errors='ignore')
    
    # Remove columns that start with 'Reporter intensity corrected' or 'TMT Channel Normal'
    df = df.loc[:, ~df.columns.str.startswith(('Reporter intensity corrected', 'TMT Channel Normal'))]
    
    return df


def pivot_long(df):
    """
    Converts a DataFrame from wide format to long format.
    This function takes a DataFrame with multiple columns representing different doses
    and pivots it to a long format where each row represents a single dose measurement.
    Args:
        df (pd.DataFrame): The input DataFrame in wide format.
    Returns:
        pd.DataFrame: The transformed DataFrame in long format with columns 'Dose' and 'Value'.
    """
    
    dose_columns = [col for col in df.columns if col.startswith('Dose')]
    
    df_long = pd.melt(
        df, 
        id_vars=[col for col in df.columns if col not in dose_columns], 
        value_vars=dose_columns, 
        var_name='Dose', 
        value_name='Value'
    )
    
    df_long['Dose'] = df_long['Dose'].str.replace('Dose ', '').astype(float)
    
    return df_long


def group_and_count_unique_time_points(df):
    """
    Groups the DataFrame based on the combined 'Cell_line' and 'Drug' columns and counts the unique 'Time' values.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing 'Cell_line', 'Drug', and 'Time' columns.
    Returns:
        pd.DataFrame: A DataFrame containing two columns:
            - 'Cell_line_Drug': The combined 'Cell_line' and 'Drug' values.
            - 'Unique Time Points': The number of unique 'Time' values for each group.
    """
    df['CL_Drug'] = df['Cell_line'] + '_' + df['Drug']
    result_df = df.groupby('CL_Drug').agg(
        Unique_Time_Points=('Time', 'nunique'),
        Unique_Time_Point_Values=('Time', lambda x: list(x.unique()))
    ).reset_index()
    result_df.columns = ['CL_Drug', 'Unique Time Points', 'Unique Time Point Values']
    
    return result_df


def filter_by_unique_time_points(df, unique_timepoints_df):
    """
    Filters the original DataFrame to keep only the rows that match the groups with more than one unique time point.
    
    Args:
        df (pd.DataFrame): The original DataFrame.
        unique_timepoints_df (pd.DataFrame): The DataFrame containing the groups and their unique time points.
    
    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    filtered_groups = unique_timepoints_df[unique_timepoints_df['Unique Time Points'] > 1]
    filtered_df = df.merge(filtered_groups[['CL_Drug']], on='CL_Drug', how='inner')
    
    return filtered_df

In [3]:
def main():
    """
    Main function to preprocess and combine data from text and TOML files.
    It searches for text and TOML files in the specified directory,
    processes the text files, updates them with corresponding TOML files, filters
    the data, and combines the results into a single DataFrame.
    
    Returns:
        pd.DataFrame: A combined DataFrame containing the processed data from all
        relevant text files.
    """
    verbose = False
    directory = '../data_decrypt'
    txt_files, toml_files = search_files(directory)
    combined_df = pd.DataFrame()
    
    # Remove time-dependent file
    if 'Rituximab_td.txt' in txt_files:
        txt_files.remove('Rituximab_td.txt')
        
    for txt_file in txt_files:
        txt_file_path = os.path.normpath(directory + '/' + txt_file)
        df = pd.read_csv(txt_file_path, delimiter='\t', header=0)
        df = process_experiment_column(df)

        toml_file_name = f"pipeline_{txt_file.replace('_dd.txt', '.toml')}"
        if toml_file_name in toml_files:
            toml_file_name = os.path.normpath(directory + '/' + toml_file_name)
            df = update_dataframe_with_toml(df, toml_file_name, verbose)
        else:
            print(f"toml file: {toml_file_name} does not exist!")
            
        min_score_cutoff = 60
        max_pep_cutoff = 0.05
        df = filter_data(df, min_score_cutoff, max_pep_cutoff, verbose)
        df = remove_columns(df)
        df = pivot_long(df)  # Dose column is transformed to nM
        
        # Fix 'Carfilzomib' Drug name to not be the same with 'Bortezomib'
        if txt_file == 'Carfilzomib_dd.txt':
            df['Drug'] = df['Drug'].replace('BTZ_CFZ', 'CFZ_BTZ')
            
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    combined_df_copy = combined_df.copy()
    unique_timepoints = group_and_count_unique_time_points(combined_df_copy)
    filtered_df = filter_by_unique_time_points(combined_df_copy, unique_timepoints)
    
    return combined_df, filtered_df, unique_timepoints

if __name__ == "__main__":
    combined_df, filtered_df, unique_timepoints = main()
    display(combined_df)

,Cell_line,Drug,Time,Modified sequence,Experiment,Gene names,Dose,Value
0,A549,PD325901,60,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,ddPTM_A549_PD325901_60min_R1,EIF3J,10000.0,NaN
1,A549,PD325901,60,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVRK,ddPTM_A549_PD325901_60min_R1,EIF3J,10000.0,1.489595
2,A549,PD325901,60,(ac)AAAADS(ph)FSGGPAGVR,ddPTM_A549_PD325901_60min_R1,RMI2,10000.0,1.307801
3,A549,MK2206,60,(ac)AAAAPDSRVS(ph)EEENLK,ddPTM_A549_MK2206_60min_R1,RRP15,10000.0,0.789177
4,A549,PD325901,60,(ac)AAAAPDSRVS(ph)EEENLK,ddPTM_A549_PD325901_60min_R1,RRP15,10000.0,1.065007
...,...,...,...,...,...,...,...,...
9998965,HeLa,SAHA,240,YYS(ph)DSDDELTVEQR,ddPTM_HeLa_SAHA_4h_R1,BOD1L1,0.0,1.000000
9998966,HeLa,SAHA,240,YYS(ph)IDDNQNK,ddPTM_HeLa_SAHA_4h_R1,NCOA7,0.0,1.000000
9998967,HeLa,SAHA,240,YYS(ph)PCEEHPAETNQNEGSESGTIR,ddPTM_HeLa_SAHA_4h_R1,ARHGEF5,0.0,1.000000
9998968,HeLa,SAHA,240,YYSDS(ph)DDELTVEQR,ddPTM_HeLa_SAHA_4h_R1,BOD1L1,0.0,1.000000


## Rituximab Time-Dependent

In [4]:
data_file_path = os.path.join("..", "data_decrypt", "rituximab_td.txt")

rituximab_td = pd.read_csv(data_file_path, sep="\t")

In [5]:
# Filter data
# Arbitrary Cutoffs
min_score_cutoff = 60  # Confidence score for peptide identification
max_pep_cutoff = 0.05  # Minimum posterior error probability
verbose = False
rows_before = rituximab_td.shape[0]

rituximab_td = rituximab_td[
    (rituximab_td['Max Score'] >= min_score_cutoff) & 
    (rituximab_td['Min PEP'] <= max_pep_cutoff) &
    (rituximab_td['Phospho (STY)'] >= 1) &
    (rituximab_td['Phosphoproteome'] == True)
]

rows_after = rituximab_td.shape[0]

if verbose == True:
    print(f"Number of rows before filtering: {rows_before}")
    print(f"Number of rows before filtering: {rows_after}")

# Break down experiment column
split_experiment_col = rituximab_td['Experiment'].str.split('_', expand=True)
rituximab_td['Cell_line'] = split_experiment_col[1]
rituximab_td['Drug'] = split_experiment_col[2]
rituximab_td['Dose'] = split_experiment_col[3].str.replace('ng', '')

In [6]:
rituximab_td = rituximab_td[[
        'Experiment',
        'Cell_line',
        'Drug',
        'Dose',
        'Gene names',
        'Modified sequence', 
        'TMT Channel Ratio 1',
        'TMT Channel Ratio 2', 
        'TMT Channel Ratio 3', 
        'TMT Channel Ratio 4',
        'TMT Channel Ratio 5', 
        'TMT Channel Ratio 6', 
        'TMT Channel Ratio 7',
        'TMT Channel Ratio 8'
]]

rituximab_td = rituximab_td.rename(columns={
        'TMT Channel Ratio 1': '1',
        'TMT Channel Ratio 2': '2',
        'TMT Channel Ratio 3': '5',
        'TMT Channel Ratio 4': '10',
        'TMT Channel Ratio 5': '60',
        'TMT Channel Ratio 6': '120',
        'TMT Channel Ratio 7': '360',
        'TMT Channel Ratio 8': '1440'})

rituximab_td.head(10)

,Experiment,Cell_line,Drug,Dose,Gene names,Modified sequence,1,2,5,10,60,120,360,1440
0,tdPTM_Ramos_Rituximab_0ng,Ramos,Rituximab,0,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1.0,1.482082,1.304435,1.200087,1.711429,1.036134,0.939321,2.293267
1,tdPTM_SU-DHL-4_Rituximab_30000ng,SU-DHL-4,Rituximab,30000,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1.0,0.936966,0.895313,0.806706,1.030389,1.129011,0.949684,1.032427
2,tdPTM_ARH-77_Rituximab_0ng,ARH-77,Rituximab,0,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.894783,0.882104,0.860725,0.996797,0.947909,0.774303,1.274237
3,tdPTM_ARH-77_Rituximab_1000ng,ARH-77,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.925674,0.996374,1.035341,0.854041,0.649231,0.801136,1.313595
4,tdPTM_ARH-77_Rituximab_30000ng,ARH-77,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tdPTM_Ramos_Rituximab_1000ng,Ramos,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.916323,1.344232,1.124650,1.409876,1.464369,1.204183,0.909416
6,tdPTM_Ramos_Rituximab_30000ng,Ramos,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,1.126545,0.926199,0.988858,1.031604,0.717043,0.922084,1.318389
7,tdPTM_SU-DHL-4_Rituximab_0ng,SU-DHL-4,Rituximab,0,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.840954,0.925568,1.176106,0.850982,0.765193,0.984658,0.934958
8,tdPTM_SU-DHL-4_Rituximab_1000ng,SU-DHL-4,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1.0,0.651056,1.080928,0.761110,0.580987,0.492847,0.519884,0.780907
9,tdPTM_SU-DHL-4_Rituximab_30000ng,SU-DHL-4,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVRK,1.0,1.003644,1.032999,1.202614,0.978781,1.083113,0.823669,0.939890


In [7]:
time_cols = ['1', '2', '5', '10', '60', '120', '360', '1440']
id_vars = ['Experiment', 'Cell_line', 'Drug', 'Dose', 'Gene names', 'Modified sequence']

rituximab_td = pd.melt(
    rituximab_td,
    id_vars=id_vars,
    value_vars=time_cols,
    var_name='Time',
    value_name='Value'
)

rituximab_td.head()

,Experiment,Cell_line,Drug,Dose,Gene names,Modified sequence,Time,Value
0,tdPTM_Ramos_Rituximab_0ng,Ramos,Rituximab,0,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1,1.0
1,tdPTM_SU-DHL-4_Rituximab_30000ng,SU-DHL-4,Rituximab,30000,PABPN1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,1,1.0
2,tdPTM_ARH-77_Rituximab_0ng,ARH-77,Rituximab,0,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1,1.0
3,tdPTM_ARH-77_Rituximab_1000ng,ARH-77,Rituximab,1000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1,1.0
4,tdPTM_ARH-77_Rituximab_30000ng,ARH-77,Rituximab,30000,EIF3J,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,1,NaN


## Combine Datasets

In [8]:
column_order = combined_df.columns
rituximab_td = rituximab_td[column_order]

combined_df = pd.concat([rituximab_td, combined_df], axis=0, ignore_index=True, sort=False)
combined_df["Gene names"] = combined_df["Gene names"].astype(str)
display(combined_df)

,Cell_line,Drug,Time,Modified sequence,Experiment,Gene names,Dose,Value
0,Ramos,Rituximab,1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,tdPTM_Ramos_Rituximab_0ng,PABPN1,0,1.0
1,SU-DHL-4,Rituximab,1,(ac)AAAAAAAAAAGAAGGRGS(ph)GPGR,tdPTM_SU-DHL-4_Rituximab_30000ng,PABPN1,30000,1.0
2,ARH-77,Rituximab,1,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,tdPTM_ARH-77_Rituximab_0ng,EIF3J,0,1.0
3,ARH-77,Rituximab,1,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,tdPTM_ARH-77_Rituximab_1000ng,EIF3J,1000,1.0
4,ARH-77,Rituximab,1,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,tdPTM_ARH-77_Rituximab_30000ng,EIF3J,30000,NaN
...,...,...,...,...,...,...,...,...
11046237,HeLa,SAHA,240,YYS(ph)DSDDELTVEQR,ddPTM_HeLa_SAHA_4h_R1,BOD1L1,0.0,1.0
11046238,HeLa,SAHA,240,YYS(ph)IDDNQNK,ddPTM_HeLa_SAHA_4h_R1,NCOA7,0.0,1.0
11046239,HeLa,SAHA,240,YYS(ph)PCEEHPAETNQNEGSESGTIR,ddPTM_HeLa_SAHA_4h_R1,ARHGEF5,0.0,1.0
11046240,HeLa,SAHA,240,YYSDS(ph)DDELTVEQR,ddPTM_HeLa_SAHA_4h_R1,BOD1L1,0.0,1.0


## Convert to VESPA Format

Create map_protein dataframe with site_id, gene_id, protein_id.

In [ ]:
uniprot_id_cache = {}
protein_seq_cache = {}
mapping_cache = {}

def map_protein_names(protein_names):
    """
    Given a set of protein names (from GeneCards identifiers),
    use unipressed to map them to UniProtKB IDs.
    """
    # Remove duplicates
    protein_names_set = set(protein_names)
    
    request = IdMappingClient.submit(
        source="GeneCards",
        dest="UniProtKB",
        ids=protein_names_set
    )
    time.sleep(5)
    results = list(request.each_result())
    mapping = {entry["from"]: entry["to"] for entry in results}
    return mapping

def get_uniprot_id(protein_name):
    """
    For a single protein name (possibly containing multiple names separated by ';'),
    use the first name to look up the UniProtKB id using our mapping cache.
    """
    # Split if there are multiple names and take the first one (heuristic method to fix multiple proteins having the same peptide sequence)
    primary_name = protein_name.split(";")[0].strip()
    if primary_name in mapping_cache:
        return mapping_cache[primary_name]
    
    mapping = map_protein_names({primary_name})
    uniprot_id = mapping.get(primary_name)
    mapping_cache[primary_name] = uniprot_id
    return uniprot_id

def get_protein_sequence(uniprot_id):
    """
    Fetch the protein sequence for a given UniProt ID using UniProt REST API.
    """
    import requests
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    response = requests.get(url)
    if response.status_code == 200:
        fasta_data = response.text
        # Remove header and join lines
        seq = "".join(fasta_data.splitlines()[1:])
        return seq
    return None

def parse_modified_sequence(mod_seq):
    """
    Parse a modified sequence (e.g., (ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR)
    to extract the unmodified peptide sequence and the 0-indexed positions 
    where the phosphorylation marker (ph) appears.
    """
    clean_seq = ""
    ph_positions = []
    i = 0  # index in clean_seq
    pattern = re.compile(r"(\([^)]+\))|([A-Za-z]+)")
    for match in pattern.finditer(mod_seq):
        if match.group(1):  # modification marker
            mod_text = match.group(1).lower()
            if mod_text == "(ph)":
                if i > 0:
                    # Mark the preceding residue as phosphorylated
                    ph_positions.append(i - 1)
        elif match.group(2):
            seq_part = match.group(2)
            clean_seq += seq_part
            i += len(seq_part)
    return clean_seq, ph_positions

def map_phosphosite(protein_seq, peptide_seq, ph_rel_positions):
    """
    Map the peptide sequence to the full protein sequence, and calculate the absolute
    (1-indexed) phosphorylation positions.
    """
    start_index = protein_seq.find(peptide_seq)
    if start_index == -1:
        return []
    abs_positions = []
    for rel_pos in ph_rel_positions:
        abs_pos = start_index + rel_pos + 1
        abs_positions.append(abs_pos)
    return abs_positions

def create_site_id(protein_name, uniprot_id, abs_positions):
    """
    Create a VESPA-style site_id string: ProteinName:UniProtID:S<pos1>[;S<pos2>;...]
    """
    if not abs_positions:
        return None
    sites = ";".join([f"S{pos}" for pos in abs_positions])
    return f"{protein_name}:{uniprot_id}:{sites}"

def process_row(row):
    """
    Process one row of the DataFrame.
    Expected columns: 'Modified sequence' and 'Gene names'.
    Returns a Series with protein_id and site_id.
    """
    mod_seq = row["Modified sequence"]
    protein_name = row["Gene names"]
    
    # Get the UniProt ID using the protein name (using the first if multiple)
    uniprot_id = get_uniprot_id(protein_name)
    if uniprot_id is None:
        return pd.Series({"protein_id": None, "site_id": None})
    
    # Fetch full protein sequence
    protein_seq = get_protein_sequence(uniprot_id)
    if protein_seq is None:
        return pd.Series({"protein_id": uniprot_id, "site_id": None})
    
    # Parse the modified sequence
    peptide_seq, ph_rel_positions = parse_modified_sequence(mod_seq)
    
    # Map the peptide to the protein sequence to get absolute phosphorylation positions
    abs_positions = map_phosphosite(protein_seq, peptide_seq, ph_rel_positions)
    
    # Create a site_id in VESPA format
    site_id = create_site_id(protein_name, uniprot_id, abs_positions)
    
    return pd.Series({"protein_id": uniprot_id, "site_id": site_id})

if __name__ == "__main__":
    gene_peptide = combined_df[['Modified sequence', 'Gene names']].drop_duplicates()
    gene_peptide['Gene names'] = gene_peptide['Gene names'].apply(lambda x: x.split(";")[0].strip())  # Fix multiple names
    map_protein = gene_peptide.apply(process_row, axis=1)
    map_protein = pd.concat([gene_peptide.reset_index(drop=True), map_protein.reset_index(drop=True)], axis=1)
    map_protein = map_protein.rename(columns={
        "Gene names": "gene_id",
        "Modified sequence": "modified_peptide_sequence"
    })
    map_protein = map_protein[['site_id', 'gene_id', 'protein_id', 'modified_peptide_sequence']]
    map_protein.to_csv("decrypt_map_protein_.csv", index=False)
    map_protein.to_csv("../data_decrypt/decrypt_map_protein.csv", index=False)


In [10]:
display(combined_df[['Modified sequence', 'Gene names']].drop_duplicates().tail(10))
display(gene_peptide.tail(10))

,Modified sequence,Gene names
10952898,YIENMS(ph)RGIYK,PHF6
10952907,YKSTT(ph)SVSEEDVSSR,PAWR
10952910,YLELMY(ph)AGAALK,ANKUB1
10952912,YLGQVT(ph)LT(ph)HKK,ZNF713
10952940,YPRFLS(ph)AEAIGIMR,PKN1
10952963,YS(ph)GRVAKEVMESSAK,FRMD5
10952966,YS(ph)PTSPTYSPTTPK,POLR2A
10952977,YSLS(ph)PSK,RANBP2;RGPD2;RGPD1;RGPD8;RGPD5;RGPD3;RGPD4
10952988,YSPTSPTYSPTSPVYT(ph)PTSPK,POLR2A
10952998,YSVLNNDDYFADVSPLRAT(ph)S(ph)PSK,KNOP1


,Modified sequence,Gene names
10952898,YIENMS(ph)RGIYK,PHF6
10952907,YKSTT(ph)SVSEEDVSSR,PAWR
10952910,YLELMY(ph)AGAALK,ANKUB1
10952912,YLGQVT(ph)LT(ph)HKK,ZNF713
10952940,YPRFLS(ph)AEAIGIMR,PKN1
10952963,YS(ph)GRVAKEVMESSAK,FRMD5
10952966,YS(ph)PTSPTYSPTTPK,POLR2A
10952977,YSLS(ph)PSK,RANBP2
10952988,YSPTSPTYSPTSPVYT(ph)PTSPK,POLR2A
10952998,YSVLNNDDYFADVSPLRAT(ph)S(ph)PSK,KNOP1
